# Authentication with DefaultAzureCredential

## Overview

DefaultAzureCredential is the recommended way to handle authentication in Azure applications. It provides a streamlined authentication flow by trying multiple credential types in sequence until one succeeds. This notebook will help you troubleshoot common authentication issues and ensure proper setup.

## Understanding DefaultAzureCredential

DefaultAzureCredential attempts authentication methods in the following order:

1. Environment Credentials
2. Workload Identity (in Kubernetes)
3. Managed Identity
4. Azure CLI Credentials
5. Azure PowerShell Credentials
6. Visual Studio Code Credentials
7. Interactive Browser Authentication (as fallback)

## Prerequisites

Ensure you have the following installed:
- Azure CLI
- Azure Developer CLI (optional)
- Required role assignments (Azure AI Developer)
- Jupyter Notebook environment - kernel configured to use Python 3.8 or later

## Authentication Methods

### 1. Using Azure CLI (Recommended for Local Development)

In [ ]:
# Install required packages
!pip install azure-identity

# First, we'll authenticate using Azure CLI
This is the recommended approach for local development.

When you run the code below, you will be redirected to:
- Either the Azure portal in your browser to complete the login 
- Or use Windows login if you're already signed in to your machine

The code will:
1. Load environment variables from .env file, including the TENANT_ID
2. Use Azure CLI to log in to your specific tenant  
3. Test authentication by attempting to get a token


In [ ]:
# Import required packages

from dotenv import load_dotenv
import os
from pathlib import Path
# Get the path to the .env file which is in the parent directory
notebook_path = Path().absolute()  # Get absolute path of current notebook
parent_dir = notebook_path.parent  # Get parent directory
load_dotenv(parent_dir / '.env')  # Load environment variables from .env file

# Get tenant ID from environment variable
tenant_id = os.getenv("TENANT_ID")
subscription_id = os.getenv("SUBSCRIPTION_ID")


# Azure login with specific tenant
!az config set core.disable_confirm_prompt=true
!az login --tenant {tenant_id} <<< "\n"

if subscription_id:
    # Set the subscription
    !az account set --subscription {subscription_id}
    print(f"✓ Successfully set subscription: {subscription_id}")
else:
    print("⚠️ Could not get subscription ID from PROJECT_CONNECTION_STRING")


# Next, we'll test the authentication by attempting to get a token using DefaultAzureCredential

The code below will:
1. Create a DefaultAzureCredential instance
2. Try to get a token for Azure Cognitive Services
3. Print success message if token is acquired

>Note: You may see some warning/error messages as it tries different authentication methods - 
>this is normal and can be ignored as long as you see "Successfully acquired token!" at the end


In [ ]:
# Then use DefaultAzureCredential in your code
from azure.identity import DefaultAzureCredential
from azure.core.credentials import AccessToken
import logging

# Enable detailed logging
logging.basicConfig(level=logging.CRITICAL)
logging.getLogger('azure.identity').setLevel(logging.CRITICAL)
logging.getLogger('azure.core').setLevel(logging.CRITICAL)

try:
    credential = DefaultAzureCredential()
    # Test token acquisition
    token = credential.get_token("https://cognitiveservices.azure.com/.default")
    print("Successfully acquired token!")
except Exception as e:
    print(f"Authentication failed: {str(e)}")